# LunAPI : staging

Links to notebooks in this repository: [Index](./00_overview.ipynb) | [Individuals](./01_indivs.ipynb) | [Projects](./02_projects.ipynb) | [Staging](./03_staging.ipynb) | [Models](./04_models.ipynb) | [Advanced](./98_advanced.ipynb) | [Reference](./99_reference.ipynb)

In [1]:
import lunapi as lp
proj = lp.proj()

initiated lunapi v0.0.4 <lunapi.lunapi0.luna object at 0xffff93d6fd70> 

enabling console outputs


In [8]:
edf = proj.inst( 0 )

In [9]:
edf

<lunapi-instance id:learn-nsrr01 edf:tutorial/edfs/learn-nsrr01.edf annot:tutorial/edfs/learn-nsrr01-profusion.xml>

In [10]:
edf.channels()

,Channels
0,SaO2
1,PR
2,EEG_sec
3,ECG
4,EMG
5,EOG_L
6,EOG_R
7,EEG
8,AIRFLOW
9,THOR_RES


In [11]:
raw = edf.edf

In [12]:
raw.channels()

['SaO2',
 'PR',
 'EEG_sec',
 'ECG',
 'EMG',
 'EOG_L',
 'EOG_R',
 'EEG',
 'AIRFLOW',
 'THOR_RES',
 'ABDO_RES',
 'POSITION',
 'LIGHT',
 'OX_STAT']

In [13]:
raw

<lunapi-instance id:learn-nsrr01 edf:tutorial/edfs/learn-nsrr01.edf annot:tutorial/edfs/learn-nsrr01-profusion.xml>

In [14]:
raw.stat()

{'annotation_files': 'tutorial/edfs/learn-nsrr01-profusion.xml',
 'duration': '11.22.00.000',
 'edf_file': 'tutorial/edfs/learn-nsrr01.edf',
 'id': 'learn-nsrr01',
 'na': 10,
 'ns': 14,
 'nt': 14,
 'state': 1}

In [15]:
edf.stat()

,Value
annotation_files,tutorial/edfs/learn-nsrr01-profusion.xml
duration,11.22.00.000
edf_file,tutorial/edfs/learn-nsrr01.edf
id,learn-nsrr01
na,10
ns,14
nt,14
state,1


In [16]:
edf.eval( 'HEADERS' )

 duration 11.22.00, 40920s | time 21.58.17 - 09.20.17 | date 01.01.85

 signals: 14 (of 14) selected in a standard EDF file
  SaO2 | PR | EEG_sec | ECG | EMG | EOG_L | EOG_R | EEG
  AIRFLOW | THOR_RES | ABDO_RES | POSITION | LIGHT | OX_STAT
 ..................................................................
 CMD #1: HEADERS
   options: sig=*
 duration 09.57.30, 35850s | time 21.18.06 - 07.15.36 | date 01.01.85

 signals: 14 (of 14) selected in a standard EDF file
  SaO2 | PR | EEG_sec | ECG | EMG | EOG_L | EOG_R | EEG
  AIRFLOW | THOR_RES | ABDO_RES | POSITION | LIGHT | OX_STAT
 ..................................................................
 CMD #1: HEADERS
   options: sig=*
 duration 11.22.00, 40920s | time 20.15.00 - 07.37.00 | date 01.01.85

 signals: 14 (of 14) selected in a standard EDF file
  SaO2 | PR | EEG_sec | ECG | EMG | EOG_L | EOG_R | EEG
  AIRFLOW | THOR_RES | ABDO_RES | POSITION | LIGHT | OX_STAT
 ..................................................................
 CM

In [17]:
edf.strata()

,Command,Stata
0,HEADERS,BL
1,HEADERS,CH


In [18]:
edf.annots()

,Annotations
0,Arousal
1,Hypopnea
2,N1
3,N2
4,N3
5,Obstructive_Apnea
6,R
7,SpO2_artifact
8,SpO2_desaturation
9,W


In [19]:
raw.channels()

['SaO2',
 'PR',
 'EEG_sec',
 'ECG',
 'EMG',
 'EOG_L',
 'EOG_R',
 'EEG',
 'AIRFLOW',
 'THOR_RES',
 'ABDO_RES',
 'POSITION',
 'LIGHT',
 'OX_STAT']

In [20]:
edf.table( 'HEADERS' )

,ID,EDF_ID,EDF_TYPE,NR,NS,NS_ALL,REC_DUR,REC_DUR_HMS,REC_DUR_SEC,START_DATE,START_TIME,STOP_TIME,TOT_DUR_HMS,TOT_DUR_SEC
0,learn-nsrr01,.,EDF,40920,14,14,1.0,11:22:00,40920.0,01.01.85,21.58.17,09.20.17,11.22.00,40920.0


In [21]:
edf.table( 'HEADERS' , 'CH' )

,ID,CH,DMAX,DMIN,PDIM,PMAX,PMIN,POS,SENS,SR,TRANS,TYPE
0,learn-nsrr01,ABDO_RES,127,-128,.,-1.0,1.0,11,-0.007843,10.0,.,EFFORT
1,learn-nsrr01,AIRFLOW,127,-128,.,-1.0,1.0,9,-0.007843,10.0,.,AIRFLOW
2,learn-nsrr01,ECG,127,-128,mV,1.25,-1.25,4,0.009804,250.0,.,ECG
3,learn-nsrr01,EEG,127,-128,uV,125.0,-125.0,8,0.980392,125.0,.,EEG
4,learn-nsrr01,EEG_sec,127,-128,uV,125.0,-125.0,3,0.980392,125.0,.,EEG
5,learn-nsrr01,EMG,127,-128,uV,31.5,-31.5,5,0.247059,125.0,.,EMG
6,learn-nsrr01,EOG_L,127,-128,uV,125.0,-125.0,6,0.980392,50.0,.,EOG
7,learn-nsrr01,EOG_R,127,-128,uV,125.0,-125.0,7,0.980392,50.0,.,EOG
8,learn-nsrr01,LIGHT,1,0,.,1.0,0.0,13,1.0,1.0,.,LIGHT
9,learn-nsrr01,OX_STAT,3,0,.,3.0,0.0,14,1.0,1.0,.,OXYGEN


In [22]:
r = edf.proc( 'HEADERS' )

 ..................................................................
 CMD #1: HEADERS
   options: sig=*



In [23]:
lp.show(r)

HEADERS: BL


,ID,EDF_ID,EDF_TYPE,NR,NS,NS_ALL,REC_DUR,REC_DUR_HMS,REC_DUR_SEC,START_DATE,START_TIME,STOP_TIME,TOT_DUR_HMS,TOT_DUR_SEC
0,learn-nsrr01,.,EDF,40920,14,14,1.0,11:22:00,40920.0,01.01.85,21.58.17,09.20.17,11.22.00,40920.0


HEADERS: CH


,ID,CH,DMAX,DMIN,PDIM,PMAX,PMIN,POS,SENS,SR,TRANS,TYPE
0,learn-nsrr01,ABDO_RES,127,-128,.,-1.0,1.0,11,-0.007843,10.0,.,EFFORT
1,learn-nsrr01,AIRFLOW,127,-128,.,-1.0,1.0,9,-0.007843,10.0,.,AIRFLOW
2,learn-nsrr01,ECG,127,-128,mV,1.25,-1.25,4,0.009804,250.0,.,ECG
3,learn-nsrr01,EEG,127,-128,uV,125.0,-125.0,8,0.980392,125.0,.,EEG
4,learn-nsrr01,EEG_sec,127,-128,uV,125.0,-125.0,3,0.980392,125.0,.,EEG
5,learn-nsrr01,EMG,127,-128,uV,31.5,-31.5,5,0.247059,125.0,.,EMG
6,learn-nsrr01,EOG_L,127,-128,uV,125.0,-125.0,6,0.980392,50.0,.,EOG
7,learn-nsrr01,EOG_R,127,-128,uV,125.0,-125.0,7,0.980392,50.0,.,EOG
8,learn-nsrr01,LIGHT,1,0,.,1.0,0.0,13,1.0,1.0,.,LIGHT
9,learn-nsrr01,OX_STAT,3,0,.,3.0,0.0,14,1.0,1.0,.,OXYGEN


In [24]:
edf.eval( 'PSD sig=EEG spectrum dB' )

 ..................................................................
 CMD #1: PSD
   options: dB sig=EEG spectrum
  set epochs to default 30 seconds, 1364 epochs
  calculating PSD from 0.5 to 25 for 1 signals



In [25]:
edf.strata()

,Command,Stata
0,PSD,B_CH
1,PSD,CH
2,PSD,CH_F


In [26]:
res = edf.table( 'PSD' , 'CH_F' ) 
res

,ID,CH,F,PSD
0,learn-nsrr01,EEG,0.5,15.937989
1,learn-nsrr01,EEG,0.75,14.770016
2,learn-nsrr01,EEG,1.0,14.334715
3,learn-nsrr01,EEG,1.25,13.391399
4,learn-nsrr01,EEG,1.5,12.118676
...,...,...,...,...
94,learn-nsrr01,EEG,24.0,-3.364087
95,learn-nsrr01,EEG,24.25,-3.416073
96,learn-nsrr01,EEG,24.5,-2.848171
97,learn-nsrr01,EEG,24.75,-2.366708


In [27]:
res = edf.proc( 'HEADERS' )

 ..................................................................
 CMD #1: HEADERS
   options: sig=*



In [28]:
res

{'HEADERS: BL':              ID EDF_ID EDF_TYPE     NR  NS NS_ALL REC_DUR REC_DUR_HMS  \
 0  learn-nsrr01      .      EDF  40920  14     14     1.0    11:22:00   
 
   REC_DUR_SEC START_DATE START_TIME STOP_TIME TOT_DUR_HMS TOT_DUR_SEC  
 0     40920.0   01.01.85   21.58.17  09.20.17    11.22.00     40920.0  ,
 'HEADERS: CH':               ID        CH   DMAX    DMIN PDIM   PMAX   PMIN POS      SENS  \
 0   learn-nsrr01  ABDO_RES    127    -128    .   -1.0    1.0  11 -0.007843   
 1   learn-nsrr01   AIRFLOW    127    -128    .   -1.0    1.0   9 -0.007843   
 2   learn-nsrr01       ECG    127    -128   mV   1.25  -1.25   4  0.009804   
 3   learn-nsrr01       EEG    127    -128   uV  125.0 -125.0   8  0.980392   
 4   learn-nsrr01   EEG_sec    127    -128   uV  125.0 -125.0   3  0.980392   
 5   learn-nsrr01       EMG    127    -128   uV   31.5  -31.5   5  0.247059   
 6   learn-nsrr01     EOG_L    127    -128   uV  125.0 -125.0   6  0.980392   
 7   learn-nsrr01     EOG_R    127    -12

In [29]:
res = proj.eval( 'HEADERS' )

In [30]:
res.keys()

dict_keys(['HEADERS: BL', 'HEADERS: CH'])

In [31]:
lp.show( res )

HEADERS: BL


,ID,EDF_ID,EDF_TYPE,NR,NS,NS_ALL,REC_DUR,REC_DUR_HMS,REC_DUR_SEC,START_DATE,START_TIME,STOP_TIME,TOT_DUR_HMS,TOT_DUR_SEC
0,learn-nsrr01,.,EDF,40920,14,14,1.0,11:22:00,40920.0,01.01.85,21.58.17,09.20.17,11.22.00,40920.0
1,learn-nsrr02,.,EDF,35850,14,14,1.0,09:57:30,35850.0,01.01.85,21.18.06,07.15.36,09.57.30,35850.0
2,learn-nsrr03,.,EDF,40920,14,14,1.0,11:22:00,40920.0,01.01.85,20.15.00,07.37.00,11.22.00,40920.0


HEADERS: CH


,ID,CH,DMAX,DMIN,PDIM,PMAX,PMIN,POS,SENS,SR,TRANS,TYPE
0,learn-nsrr01,ABDO_RES,127,-128,.,-1.0,1.0,11,-0.007843,10.0,.,EFFORT
1,learn-nsrr02,ABDO_RES,127,-128,.,-1.0,1.0,11,-0.007843,10.0,.,EFFORT
2,learn-nsrr03,ABDO_RES,127,-128,.,-1.0,1.0,11,-0.007843,10.0,.,EFFORT
3,learn-nsrr01,AIRFLOW,127,-128,.,-1.0,1.0,9,-0.007843,10.0,.,AIRFLOW
4,learn-nsrr02,AIRFLOW,127,-128,.,-1.0,1.0,9,-0.007843,10.0,.,AIRFLOW
5,learn-nsrr03,AIRFLOW,127,-128,.,-1.0,1.0,9,-0.007843,10.0,.,AIRFLOW
6,learn-nsrr01,ECG,127,-128,mV,1.25,-1.25,4,0.009804,250.0,.,ECG
7,learn-nsrr02,ECG,127,-128,mV,1.25,-1.25,4,0.009804,250.0,.,ECG
8,learn-nsrr03,ECG,127,-128,mV,1.25,-1.25,4,0.009804,250.0,.,ECG
9,learn-nsrr01,EEG,127,-128,uV,125.0,-125.0,8,0.980392,125.0,.,EEG


To load and parse (i.e. splicing in `&` characters between commands and removing comments starting `%`) a Luna script from a file, use `lp.cmdfile()`:

In [32]:
lp.cmdfile( 'tutorial/cmd/first.txt') 

'EPOCH len=30 & TAG tag=STAGE/${stage} & MASK ifnot=${stage} & RESTRUCTURE & STATS sig=EEG'

In [33]:
proj.import_db( 'out.db' ) 

[]

In [34]:
proj.commands()

,Command
0,HEADERS


In [35]:
proj.table( 'HEADERS' )

,ID,EDF_ID,EDF_TYPE,NR,NS,NS_ALL,REC_DUR,REC_DUR_HMS,REC_DUR_SEC,START_DATE,START_TIME,STOP_TIME,TOT_DUR_HMS,TOT_DUR_SEC
0,learn-nsrr01,.,EDF,40920,14,14,1.0,11:22:00,40920.0,01.01.85,21.58.17,09.20.17,11.22.00,40920.0
1,learn-nsrr02,.,EDF,35850,14,14,1.0,09:57:30,35850.0,01.01.85,21.18.06,07.15.36,09.57.30,35850.0
2,learn-nsrr03,.,EDF,40920,14,14,1.0,11:22:00,40920.0,01.01.85,20.15.00,07.37.00,11.22.00,40920.0


In [36]:
res = proj.eval( 'STATS' )

In [37]:
lp.show( res ) 

STATS: CH


,ID,CH,KURT,MAX,MEAN,MIN,P01,P02,P05,P10,...,P60,P70,P80,P90,P95,P98,P99,RMS,SD,SKEW
0,learn-nsrr01,ABDO_RES,5.26486,0.992157,0.021829,-1.0,-0.709804,-0.505882,-0.317647,-0.2,...,0.027451,0.05098,0.098039,0.262745,0.435294,0.662745,0.87451,0.235721,0.234708,0.334763
1,learn-nsrr02,ABDO_RES,-0.725851,0.05098,0.024713,-0.011765,0.019608,0.019608,0.019608,0.019608,...,0.027451,0.027451,0.027451,0.027451,0.027451,0.027451,0.027451,0.025009,0.003832,-0.563866
2,learn-nsrr03,ABDO_RES,2.218339,0.992157,-0.010816,-1.0,-1.0,-1.0,-0.772549,-0.388235,...,-0.011765,0.011765,0.105882,0.396078,0.835294,0.992157,0.992157,0.376678,0.376523,0.128351
3,learn-nsrr01,AIRFLOW,20.490936,0.992157,0.066442,-1.0,-0.2,-0.129412,-0.066667,-0.027451,...,0.07451,0.090196,0.113725,0.160784,0.207843,0.278431,0.34902,0.121956,0.102268,0.138384
4,learn-nsrr02,AIRFLOW,1.163038,0.992157,-0.028178,-1.0,-0.513725,-0.443137,-0.356863,-0.278431,...,-0.019608,0.035294,0.121569,0.239216,0.32549,0.411765,0.47451,0.201189,0.199206,0.121003
5,learn-nsrr03,AIRFLOW,1.944107,0.992157,-0.045175,-1.0,-0.47451,-0.419608,-0.341176,-0.262745,...,-0.035294,-0.019608,0.05098,0.184314,0.286275,0.396078,0.466667,0.18273,0.177058,0.326054
6,learn-nsrr01,ECG,13.984303,1.25,0.009396,-1.240196,-1.240196,-0.857843,-0.191176,-0.083333,...,0.02451,0.034314,0.034314,0.083333,0.210784,0.828431,1.25,0.266661,0.266495,-0.115245
7,learn-nsrr02,ECG,8.922235,1.25,0.005671,-1.240196,-0.779412,-0.426471,-0.29902,-0.240196,...,0.053922,0.073529,0.083333,0.142157,0.357843,0.965686,1.132353,0.274497,0.274438,0.790402
8,learn-nsrr03,ECG,11.165007,1.25,0.003571,-1.240196,-1.240196,-1.240196,-0.259804,-0.142157,...,0.02451,0.034314,0.063725,0.142157,0.259804,1.25,1.25,0.30003,0.300009,-0.094024
9,learn-nsrr01,EEG,4.657487,125.0,-0.301199,-124.019608,-124.019608,-124.019608,-70.098039,-24.019608,...,2.45098,4.411765,8.333333,23.039216,72.058824,125.0,125.0,37.801351,37.800154,0.086212


In [38]:
res = proj.eval( 'PSD sig=EEG spectrum dB' )

In [39]:
lp.show(res)

PSD: B_CH


,ID,B,CH,PSD,RELPSD
0,learn-nsrr01,ALPHA,EEG,18.485995,0.053786
1,learn-nsrr02,ALPHA,EEG,17.306002,0.034517
2,learn-nsrr03,ALPHA,EEG,22.360021,0.063773
3,learn-nsrr01,BETA,EEG,23.350128,0.164848
4,learn-nsrr02,BETA,EEG,20.108489,0.065808
5,learn-nsrr03,BETA,EEG,24.834899,0.112751
6,learn-nsrr01,DELTA,EEG,22.203296,0.12659
7,learn-nsrr02,DELTA,EEG,26.366171,0.278003
8,learn-nsrr03,DELTA,EEG,27.058942,0.188159
9,learn-nsrr01,GAMMA,EEG,16.564119,0.034553


PSD: CH


,ID,CH,NE
0,learn-nsrr01,EEG,1364
1,learn-nsrr02,EEG,1195
2,learn-nsrr03,EEG,1364


PSD: CH_F


,ID,CH,F,PSD
0,learn-nsrr01,EEG,0.5,15.937989
1,learn-nsrr02,EEG,0.5,20.698263
2,learn-nsrr03,EEG,0.5,21.647401
3,learn-nsrr01,EEG,0.75,14.770016
4,learn-nsrr02,EEG,0.75,20.440087
...,...,...,...,...
292,learn-nsrr02,EEG,24.75,-1.735901
293,learn-nsrr03,EEG,24.75,1.567123
294,learn-nsrr01,EEG,25.0,-0.901578
295,learn-nsrr02,EEG,25.0,0.329983


In [40]:
res = proj.eval( 'ANNOTS' ) 

In [41]:
proj.strata()

,Command,Stata
0,ANNOTS,ANNOT
1,ANNOTS,ANNOT_INST
2,ANNOTS,ANNOT_INST_T1_T2


In [42]:
%%sh
ls

Dockerfile
Dockerfile~
example
example2.cpp
example3.cpp
example.cpp
example.cpp~
example-lgbm.cpp
learn-nsrr01.edf
learn-nsrr01-profusion.xml
libluna.a
libz.a
ll
pybuild-notes.org
README.md
README.md~
tutorial
tutorial.ipynb
Untitled1.ipynb
Untitled2.ipynb
Untitled3.ipynb
Untitled4.ipynb
Untitled5.ipynb
Untitled Folder
Untitled.ipynb


In [43]:
edf

<lunapi-instance id:learn-nsrr01 edf:tutorial/edfs/learn-nsrr01.edf annot:tutorial/edfs/learn-nsrr01-profusion.xml>

In [44]:
edf.stat()

,Value
annotation_files,tutorial/edfs/learn-nsrr01-profusion.xml
duration,11.22.00.000
edf_file,tutorial/edfs/learn-nsrr01.edf
elen,30.0
id,learn-nsrr01
na,10
ne,1364
nem,0
ns,14
nt,14


In [45]:
edf.annots()

,Annotations
0,Arousal
1,Hypopnea
2,N1
3,N2
4,N3
5,Obstructive_Apnea
6,R
7,SpO2_artifact
8,SpO2_desaturation
9,W


In [46]:
p = proj.inst(1)

In [55]:
res = edf.proc( 'COPY sig=EEG tag=NORM & POPS path=/build/nsrr/common/resources/pops/ lib=s2  alias=CEN,ZEN|EEG,EEG_NORM' ) 

RuntimeError: could not find CEN (or any specified aliases)